Take an i2i-model where all labels used for training are known.

generate a set of translated easy_images using only labels used for training, generate another set using only labels not used in training.

calculate FID between these two sets; calculate FID for each set and real easy_samples

In [1]:
# add parent dir to syspath
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
#root_dir = os.path.dirname(parent_dir)
sys.path.insert(0, parent_dir)

In [2]:
from pathlib import Path
import pickle
import numpy as np
from functools import reduce
from matplotlib import pyplot as plt
from imageGenerators import load_realdata
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE

from imageGenerators.imgGen_augmented import synth_generator as synth_generator_new
from models.cyclegan_modified.cyclegan import cyclegan
from FID import FID_interface

In [3]:
# model to test
i2i_path = Path("D:/m2/savedmodels/goals/q5_lambda/n512_lambda10_2")
i2i_epoch = 2

# number of images to generate for each imageset
n_images = 1000
n_digits = 5

In [4]:
# dimensions of input to wmn-model
fcsrn_height = 48; fcsrn_width = 160; fcsrn_channels = 3
fcsrn_shape = (fcsrn_height, fcsrn_width, fcsrn_channels)
fcsrn_dims = (fcsrn_width, fcsrn_height)
# get shape of inputs for i2i-model
inputshapePath = i2i_path / "inputshape"
i2i_shape = [int(s) for s in inputshapePath.read_text().split(",")]
i2i_dims = (i2i_shape[1], i2i_shape[0])

In [5]:
# load labels used for training
labels_path = i2i_path / "training_labels.pickle"
with open(labels_path, "rb") as picklefile:
    labels_synthetic, labels_real = pickle.load(picklefile)
# append labels
train_labels = np.append(labels_synthetic, labels_real, axis=0)
print(train_labels.shape)

(1024, 5)


In [6]:
# transforms label(1d-array of ints) into a string.
# used to make a map of labels for quick lookup.
def label2string(label):
    label = [str(digit)+"," for digit in label]
    return reduce(lambda state,digit: state+digit, label)

In [7]:
# map with traininglabels as keys 
labelString_map = [label2string(label) for label in train_labels]
labelString_map = dict.fromkeys(labelString_map)

# check whether label is in trainset
def label_is_in_vocab(label):
    s = label2string(label)
    return s in labelString_map

# assert every label of a labelset is contained in trainset
def assert_labelsInVocab(labels):
    labels = [label2string(label) for label in labels]
    labels = [label in labelString_map for label in labels]
    np.testing.assert_array_equal(labels, True)
# assert every label of a labelset is not contained in trainset
def assert_labelsNotInVocab(labels):
    labels = [label2string(label) for label in labels]
    labels = [label in labelString_map for label in labels]
    np.testing.assert_array_equal(labels, False)

In [8]:
labels_in_vocab = train_labels[np.random.randint(0,len(train_labels), (n_images) )]
assert_labelsInVocab(labels_in_vocab)

labels_notIn_vocab = np.random.randint(0,20, (n_images, n_digits) )
for i in range(n_images):
    iterations = 0
    while label_is_in_vocab(labels_notIn_vocab[i]):
        assert iterations < 1000, "could not find random label not in vocab."
        labels_notIn_vocab[i] = np.random.randint(0,20, (n_digits) )
assert_labelsNotInVocab(labels_notIn_vocab)

In [9]:
# generate synthetic images
synthGenerator = synth_generator_new()
synth_inVocab = synthGenerator.makeImages(labels_in_vocab, resizeTo=i2i_dims, color=True, rotate=True)
synth_outsideVocab = synthGenerator.makeImages(labels_notIn_vocab, resizeTo=i2i_dims, color=True, rotate=True)

# load model, translate synthethic images
cgModel = cyclegan(image_shape=i2i_shape,
                   n_images=0,
                   batchsize=1,
                   adversial_loss="bce",
                   lr=0,
                   _lambda=0,
                   poolsize=0,
                   checkpoint_path=i2i_path,
                   load_checkpoint_after_epoch=i2i_epoch
                  )



loaded checkpoint:  D:\m2\savedmodels\goals\q5_lambda\n512_lambda10_2\epoch-2


In [10]:
def translate_images(synthImages, i2i_model):
    model_input = tf.data.Dataset.from_tensor_slices(synthImages)\
                .map(i2i_model.preprocess_input, num_parallel_calls=AUTOTUNE)\
                .cache()\
                .batch(1)
    ####
    # predict realistic images
    translated_images = i2i_model.gen_AtoB.predict(model_input)
    # denormalize
    translated_images = (translated_images + 1) * 127.5
    return translated_images

generated_inVocab = translate_images(synth_inVocab, cgModel)
generated_outsideVocab = translate_images(synth_outsideVocab, cgModel)

In [11]:
def showImage(image):
    plt.imshow(image[:,:,0], cmap="gray", vmin=-1,vmax=1)
    plt.show()
def showIntImage(image):
    plt.imshow(image, cmap="gray", vmin=0,vmax=255)
    plt.show()

In [12]:
# load real easy samples
images_easy, labels_easy = load_realdata.load_wmr_easy(n_toLoad=n_images, resizeTo=i2i_dims)

C:\Users\andre\jupyter_ws\imageGenerators\load_realdata.py:57: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(txt_path, sep=seperators ,header=None)


In [13]:
# calculate FID-scores
inVocab_stats = FID_interface.calculate_stats(generated_inVocab)
outsideVocab_stats = FID_interface.calculate_stats(generated_outsideVocab)
real_stats = FID_interface.calculate_stats(images_easy)

fid_in_Real = FID_interface.calculate_fid_from_stats(inVocab_stats, real_stats)
fid_outside_Real = FID_interface.calculate_fid_from_stats(outsideVocab_stats, real_stats)
fid_in_outside = FID_interface.calculate_fid_from_stats(inVocab_stats, outsideVocab_stats)

print("fid(in,real:\t%d)" % (fid_in_Real) ) 
print("fid(out,real:\t%d)" % (fid_outside_Real) ) 
print("fid(in,out:\t%d)" % (fid_in_outside) ) 

fid(in,real:	90)
fid(out,real:	95)
fid(in,out:	13)
